In [1]:
import pandas as pd
import numpy as np

def wraptable(thing, caption):
    # replace ascii bs
    thing = thing.replace("\\textasciicircum \\{a\\}", "\\textsuperscript{a}").replace("\\textasciicircum \\{b\\}", "\\textsuperscript{b}" )
    for substr in ['MUTAG', 'ER\\textsuperscript{a}', 'BA\\textsuperscript{a}', 'HK\\textsuperscript{a}', 'WC\\textsuperscript{a}', 'ForcedRB &']:
        thing = thing.replace(substr, f'\midrule \n{substr}', 1)
    thing = thing.replace("@@@", "")
    thing = thing.replace("Nmin", "$N_\\text{min}$")
    thing = thing.replace("Nmax", "$N_\\text{max}$")
    thing = thing.replace("gurobi", 'Gurobi')
    thing = thing.replace("LiftMP", 'OptGNN')
    replace = {'REDDIT-BINARY': '\midrule \nREDDIT-BIN\\textsuperscript{c}',
           'REDDIT-MULTI-12K': 'REDDIT-M-12K\\textsuperscript{c}',
            'REDDIT-MULTI-5K': 'REDDIT-M-5K\\textsuperscript{c}',
            'gen\\_n': "$N_\\text{min}$",
            'greedy': 'Greedy',
            'Type': 'Dataset',
            'dataset': 'Dataset'} 
    for k, v in replace.items():
        thing = thing.replace(k, v)
    return thing
    #return "\\begin{table} \n" + thing + f"\n\\caption{{ {caption}}} \n" + "\\end{table}\n"
    

In [2]:
fname = 'Table1_maxcut.csv'
fname = 'Table1_VC.csv'
df = pd.read_csv(fname)
df = df[~df.dataset.isin(['RANDOM', 'ForcedRB'])]
dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
                              ['BA^{a}', 'ER^{a}', 'HK^{a}', 'WC^{a}', 'MUTAG^{b}', 'ENZYMES^{b}', 'PROTEINS^{b}',   'IMDB-BIN^{b}', 'COLLAB^{b}',]))

df.dataset = df.dataset.map(lambda x: dataset_conversion.get(x, x))
#df = pd.read_csv('Table1_VC.csv')

In [3]:
gen_n_dict = dict(zip([50, 100, 400, "", np.nan], [100, 200, 500, "", ""]))
nudf = df.round(2).fillna("")
nudf.gen_n = nudf.gen_n.apply(lambda x: int(x) if x != "" else x)
nudf['Nmax'] = nudf.gen_n.apply(lambda x: gen_n_dict[x])
nudf['Nmin'] = nudf.gen_n
nudf.dataset = nudf.apply(lambda row: f'{row.dataset} ({row.Nmin},{row.Nmax})', axis=1)
nudf.dataset = nudf.dataset.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)

nudf = nudf[['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP']]
main_body = (nudf.to_latex(index=False))
print(wraptable(main_body,fname))

\begin{tabular}{llllll}
\toprule
         Dataset &           GAT &          GCNN &           GIN &     GatedGCNN &        OptGNN \\
\midrule
 \midrule 
ER\textsuperscript{a} (50,100) &  58.74 (0.02) &  64.26 (0.02) &  63.50 (0.02) &  56.17 (0.02) &  55.25 (0.02) \\
ER\textsuperscript{a} (100,200) & 129.47 (0.02) & 141.94 (0.02) & 140.06 (0.02) & 131.38 (0.02) & 126.52 (0.02) \\
ER\textsuperscript{a} (400,500) & 443.94 (0.02) & 444.12 (0.02) & 442.11 (0.02) & 440.90 (0.02) & 420.70 (0.02) \\
       \midrule 
MUTAG\textsuperscript{b} &   7.79 (0.02) &   8.11 (0.02) &  17.16 (0.02) &   7.74 (0.02) &   7.74 (0.02) \\
     ENZYMES\textsuperscript{b} &  21.82 (0.02) &  25.42 (0.02) &  32.93 (0.02) &  20.18 (0.02) &  20.00 (0.02) \\
    PROTEINS\textsuperscript{b} &  27.77 (0.02) &  31.07 (0.02) &  41.32 (0.02) &  25.25 (0.02) &  25.29 (0.02) \\
    IMDB-BIN\textsuperscript{b} &  17.62 (0.02) &  19.20 (0.02) &  20.02 (0.02) &  16.78 (0.02) &  16.76 (0.02) \\
      COLLAB\textsuperscript{b} &

/tmp/ipykernel_1592801/3831217918.py:10: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = (nudf.to_latex(index=False))


In [4]:
nudf.keys()

Index(['dataset', 'GAT', 'GCNN', 'GIN', 'GatedGCNN', 'LiftMP'], dtype='object')

# Table 2

In [5]:
fname = 'Table2_maxcut.csv'
#fname = 'Table2_VC.csv'

df = pd.read_csv(fname)
df = df[~df.Type.isin(['RANDOM', 'ForcedRB'])]

#dataset_conversion = dict(zip(['BarabasiAlbert', 'ErdosRenyi', 'PowerlawCluster', 'WattsStrogatz', 'MUTAG', 'ENZYMES', 'PROTEINS',   'IMDB-BINARY', 'COLLAB',], 
#                              ['BA^{b}', 'ER^{b}', 'PC^{b}', 'WC^{b}', 'MUTAG^{a}', 'ENZYMES^{a}', 'PROTEINS^{a}',   'IMDB-BINARY^{a}', 'COLLAB^{a}',]))

df.Type = df.Type.map(lambda x: dataset_conversion.get(x, x))#df = pd.read_csv('Table2_VC.csv')


In [6]:
nudf = df.iloc[:,1:].round(2).fillna("")
#
#print(nudf.keys())



nudf.Nmin = nudf.Nmin.apply(lambda x: int(x) if x!="" else x)
nudf.Nmax = nudf.Nmax.apply(lambda x: int(x) if x!="" else x)
nudf.Type = nudf.apply(lambda row: f'{row.Type} ({row.Nmin},{row.Nmax})', axis=1)
nudf.Type = nudf.Type.apply(lambda dataset: dataset[:-4] if dataset.endswith("(,)") else dataset)
#nudf['dataset'] = nudf.Type
nudf = nudf[[x for x in nudf.keys() if x not in ['Nmin', 'Nmax'
]]]

cols = nudf.columns
hrow1 = []
hrow2 = []
for x in cols:
    if x.startswith('gurobi'):
        hrow1.append("gurobi")
        hrow2.append(x.split("_")[1] + "s")
    else:
        hrow1.append(x)
        hrow2.append("@@@")

header = pd.MultiIndex.from_arrays([hrow1, hrow2])
nudf.columns = header

main_body = nudf.to_latex(index=False)

print(wraptable(main_body,fname))



\begin{tabular}{llrrrrrrr}
\toprule
            Dataset &          OptGNN & Greedy & \multicolumn{6}{l}{Gurobi} \\
              &              &     &    0.1s &    0.5s &    1.0s &    2.0s &    4.0s &    8.0s \\
\midrule
 \midrule 
BA\textsuperscript{a} (50,100) &   351.49 (0.02) &  200.1 &   351.9 &   352.1 &   352.1 &   352.1 &   352.1 &   352.1 \\
BA\textsuperscript{a} (100,200) &   717.19 (0.02) &  408.0 &   719.4 &   719.5 &   719.7 &   719.8 &   719.9 &   720.2 \\
BA\textsuperscript{a} (400,500) &  2198.11 (0.02) & 1255.2 &  2208.1 &  2208.1 &  2208.1 &  2208.6 &  2212.4 &  2212.5 \\
 \midrule 
ER\textsuperscript{a} (50,100) &   529.00 (0.02) &  298.5 &   529.9 &   530.0 &   530.0 &   530.1 &   530.1 &   530.2 \\
ER\textsuperscript{a} (100,200) &  1995.05 (0.02) & 1097.3 &  2002.9 &  2002.9 &  2002.9 &  2002.9 &  2002.9 &  2002.9 \\
ER\textsuperscript{a} (400,500) & 16387.46 (0.02) & 8496.8 & 16476.7 & 16481.6 & 16491.6 & 16494.4 & 16495.1 & 16495.3 \\
 \midrule 
HK\textsuperscr

/tmp/ipykernel_1592801/2007938396.py:29: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  main_body = nudf.to_latex(index=False)


# instructions: 
2. add midrules